In [33]:
# Load required libraries

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import greatest
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pyspark.sql.functions import udf, col, lower, regexp_replace
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from nltk.stem.snowball import SnowballStemmer
#spark ML imports
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StringIndexer, CountVectorizer, IDF
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import VectorAssembler
%matplotlib inline

In [2]:
#create Spark session
spark = SparkSession.builder.appName('Final_project').getOrCreate()

#change configuration settings on Spark 
conf = spark.sparkContext._conf.setAll([('spark.executor.memory', '5g'), ('spark.app.name', 'Spark Updated Conf'), ('spark.executor.cores', '4'), ('spark.cores.max', '4'), ('spark.driver.memory','8g')])

sc = spark.sparkContext

In [3]:
from pyspark.sql import HiveContext
hive_context = HiveContext(sc)

In [4]:
df = hive_context.table("akarshsahu.processed_dataset")

In [51]:
df.limit(2).toPandas()

,Movie_Id,Cust_Id,Rating,tconst,Year,Name,primaryTitle,titleType,runtimeMinutes,startYear,...,Actor_6,Actor_7,Actor_8,Actor_9,Director_0,Director_1,Director_2,Director_3,Director_4,Director_5
0,7706,770129,4.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,None,None,None,Lawrence Kasdan,None,None,None,None,None
1,7706,1931185,5.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,None,None,None,Lawrence Kasdan,None,None,None,None,None


In [16]:
df.printSchema()

root
 |-- Movie_Id: integer (nullable = true)
 |-- Cust_Id: integer (nullable = true)
 |-- Rating: double (nullable = true)
 |-- tconst: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- runtimeMinutes: double (nullable = true)
 |-- startYear: integer (nullable = true)
 |-- isAdult: integer (nullable = true)
 |-- genres: string (nullable = true)
 |-- averageRating: double (nullable = true)
 |-- numVotes: integer (nullable = true)
 |-- Actor_0: string (nullable = true)
 |-- Actor_1: string (nullable = true)
 |-- Actor_2: string (nullable = true)
 |-- Actor_3: string (nullable = true)
 |-- Actor_4: string (nullable = true)
 |-- Actor_5: string (nullable = true)
 |-- Actor_6: string (nullable = true)
 |-- Actor_7: string (nullable = true)
 |-- Actor_8: string (nullable = true)
 |-- Actor_9: string (nullable = true)
 |-- Director_0: string (nullable = tr

In [52]:
#Split genres by comma
tag_split = F.split(df['genres'], ",")
df = df.withColumn('genres_split', tag_split)
df.select('genres', 'genres_split').show(10)

+------------------+--------------------+
|            genres|        genres_split|
+------------------+--------------------+
|Action,Crime,Drama|[Action, Crime, D...|
|Action,Crime,Drama|[Action, Crime, D...|
|Action,Crime,Drama|[Action, Crime, D...|
|Action,Crime,Drama|[Action, Crime, D...|
|Action,Crime,Drama|[Action, Crime, D...|
|Action,Crime,Drama|[Action, Crime, D...|
|Action,Crime,Drama|[Action, Crime, D...|
|Action,Crime,Drama|[Action, Crime, D...|
|Action,Crime,Drama|[Action, Crime, D...|
|Action,Crime,Drama|[Action, Crime, D...|
+------------------+--------------------+
only showing top 10 rows



In [53]:
# We will explode the columns to get count of unique tags

from pyspark.sql.functions import explode
from pyspark.sql.functions import trim, col

tag_counts = df.select(explode('genres_split').alias('genres_split')) \
                    .withColumn('genres_split', trim(col('genres_split'))) \
                    .groupby('genres_split') \
                    .count() \
                    .orderBy('count', ascending = False).toPandas()

In [54]:
tag_counts

,genres_split,count
0,Drama,41755335
1,Comedy,31446564
2,Action,20343640
3,Crime,16401315
4,Romance,16247175
5,Adventure,13957764
6,Thriller,12954014
7,Mystery,7206974
8,Fantasy,6213506
9,Sci-Fi,6208678


In [24]:
df.limit(2).toPandas()

,Movie_Id,Cust_Id,Rating,tconst,Year,Name,primaryTitle,titleType,runtimeMinutes,startYear,...,Actor_7,Actor_8,Actor_9,Director_0,Director_1,Director_2,Director_3,Director_4,Director_5,genres_split
0,7706,770129,4.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,None,None,Lawrence Kasdan,None,None,None,None,None,"[Action, Crime, Drama]"
1,7706,1931185,5.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,None,None,Lawrence Kasdan,None,None,None,None,None,"[Action, Crime, Drama]"


In [55]:
# Remove \\N from genres
tags = list(tag_counts.iloc[:,0])
tags = [i for i in tags if i != '\\N']

In [57]:
# Adding dummy variables for each tags
for tag in tags:
    df = df.withColumn(tag, col('genres').contains(tag).cast('integer'))

In [68]:
# Create engineered genres
from pyspark.sql.functions import greatest
newdf = df.withColumn('Act_Cri_Thr_Myst_Hor', greatest(df['Action'], df['Crime'], df['Thriller'], df['Mystery'], df['Horror']))
newdf = newdf.withColumn('Ad_Fan_Sci_Ani', greatest(df['Adventure'], df['Fantasy'], df['Sci-Fi'], df['Animation']))
newdf = newdf.withColumn('Bio_War_Hist_Doc', greatest(df['Biography'], df['War'], df['History'], df['Documentary']))
newdf = newdf.withColumn('Mus_Musi_Fam', greatest(df['Music'], df['Musical'], df['Family']))
newdf = newdf.withColumn('Noir_West', greatest(df['Film-Noir'], df['Western']))
newdf = newdf.withColumn('Ad_Rom', greatest(df['Adult'], df['Romance']))
newdf = newdf.withColumn('Drama_n', greatest(df['Drama'], df['Sport']))
newdf = newdf.withColumn('Comedy_n', df['Comedy'])

In [77]:
#Drop original genres
newdf = newdf.drop(*tags)

In [78]:
newdf.columns

['Movie_Id',
 'Cust_Id',
 'Rating',
 'tconst',
 'Year',
 'Name',
 'primaryTitle',
 'titleType',
 'runtimeMinutes',
 'startYear',
 'isAdult',
 'genres',
 'averageRating',
 'numVotes',
 'Actor_0',
 'Actor_1',
 'Actor_2',
 'Actor_3',
 'Actor_4',
 'Actor_5',
 'Actor_6',
 'Actor_7',
 'Actor_8',
 'Actor_9',
 'Director_0',
 'Director_1',
 'Director_2',
 'Director_3',
 'Director_4',
 'Director_5',
 'genres_split',
 'Act_Cri_Thr_Myst_Hor',
 'Ad_Fan_Sci_Ani',
 'Bio_War_Hist_Doc',
 'Mus_Musi_Fam',
 'Noir_West',
 'Ad_Rom',
 'Drama_n',
 'Comedy_n']

In [79]:
newdf.limit(10).toPandas()

,Movie_Id,Cust_Id,Rating,tconst,Year,Name,primaryTitle,titleType,runtimeMinutes,startYear,...,Director_5,genres_split,Act_Cri_Thr_Myst_Hor,Ad_Fan_Sci_Ani,Bio_War_Hist_Doc,Mus_Musi_Fam,Noir_West,Ad_Rom,Drama_n,Comedy_n
0,7706,770129,4.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0
1,7706,1931185,5.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0
2,7706,250166,5.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0
3,7706,328654,3.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0
4,7706,2414873,4.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0
5,7706,1239283,2.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0
6,7706,1057021,4.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0
7,7706,2633985,3.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0
8,7706,2419562,5.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0
9,7706,1047788,2.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0


In [80]:
## Write to Hive tables

newdf.write.mode('overwrite').saveAsTable("akarshsahu.processed_dataset_v2")
newdf.write.mode('overwrite').saveAsTable("aghose.processed_dataset_v2")
newdf.write.mode('overwrite').saveAsTable("mwehr.processed_dataset_v2")

In [18]:
## Read the hive table back

df1 = hive_context.table("akarshsahu.processed_dataset_v2")

In [19]:
df1.limit(2).toPandas()

,Movie_Id,Cust_Id,Rating,tconst,Year,Name,primaryTitle,titleType,runtimeMinutes,startYear,...,Director_5,genres_split,Act_Cri_Thr_Myst_Hor,Ad_Fan_Sci_Ani,Bio_War_Hist_Doc,Mus_Musi_Fam,Noir_West,Ad_Rom,Drama_n,Comedy_n
0,7706,770129,4.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0
1,7706,1931185,5.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0


In [7]:
#Load top 100 actors/actresses and directors
actor = list(spark.read.csv("/user/akarshsahu/data/Netflix_Data/Top_Actors.csv", header = True).toPandas().iloc[:,0])
director = list(spark.read.csv("/user/akarshsahu/data/Netflix_Data/Top_Directors.csv", header = True).toPandas().iloc[:,0])

In [118]:
actor

['Jack Nicholson',
 'Charlton Heston',
 'Humphrey Bogart',
 'James Stewart',
 'Lorne Greene',
 'Michael Landon',
 'Tom Hanks',
 'Clint Eastwood',
 'Al Pacino',
 'James Arness',
 'Henry Fonda',
 'Sidney Poitier',
 'Jim Carrey',
 'Michael Douglas',
 'Kirk Douglas',
 'Rita Hayworth',
 'Shirley Temple',
 'William Devane',
 'Gregory Peck',
 'Elizabeth Taylor',
 'Barbara Stanwyck',
 'Judy Garland',
 'Dennis Quaid',
 'Pernell Roberts',
 'Anthony Hopkins',
 'Patrick Swayze',
 'Mickey Rooney',
 'Larry Hagman',
 'Barbara Eden',
 'Farrah Fawcett',
 'Kevin Costner',
 'Will Smith',
 'Robin Williams',
 'Orson Welles',
 'Cary Grant',
 'Gary Cooper',
 'Laurence Olivier',
 'Chish� Ry�',
 'Peter Lawford',
 'Burt Lancaster',
 'Edmund Gwenn',
 'Donna Mills',
 'Buddy Ebsen',
 'Lucille Ball',
 'Doris Day',
 'Paula Prentiss',
 'Marilyn Monroe',
 'Bette Davis',
 'Joan Crawford',
 'John Wayne',
 'Will Geer',
 'Michael Learned',
 'Bill Bixby',
 'Sylvester Stallone',
 'Charles Bronson',
 'Michael Keaton',
 'Jame

In [90]:
director.show()

+--------------------+
|           DIRECTORS|
+--------------------+
|         David Lynch|
|     Stanley Kubrick|
|      Robert Bresson|
|    Alfred Hitchcock|
|     Martin Scorsese|
|      Clint Eastwood|
|       William Wyler|
|        Billy Wilder|
|           John Ford|
|          Fritz Lang|
|      Fred Zinnemann|
|        Sidney Lumet|
|Francis Ford Coppola|
|       David Fincher|
|    Vittorio De Sica|
|        Sergio Leone|
|    Federico Fellini|
|      Ingmar Bergman|
|          Elia Kazan|
|         John Huston|
+--------------------+
only showing top 20 rows



In [11]:
df1.limit(3).toPandas()

,Movie_Id,Cust_Id,Rating,tconst,Year,Name,primaryTitle,titleType,runtimeMinutes,startYear,...,Director_5,genres_split,Act_Cri_Thr_Myst_Hor,Ad_Fan_Sci_Ani,Bio_War_Hist_Doc,Mus_Musi_Fam,Noir_West,Ad_Rom,Drama_n,Comedy_n
0,7706,770129,4.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0
1,7706,1931185,5.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0
2,7706,250166,5.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,None,"[Action, Crime, Drama]",1,0,0,0,0,0,1,0


In [20]:
# Generate Fields - TopActors
for i in range(10):
    df1 = df1.withColumn("TopActor_" + str(i), col('Actor_' + str(i)).isin(actor).cast('integer'))


In [21]:
# Generate Fields - Directors
for i in range(6):
    df1 = df1.withColumn("TopDirector_" + str(i), col('Director_' + str(i)).isin(director).cast('integer'))


In [30]:
act_col = ["TopActor_" + str(i) for i in range(10)]
dir_col = ["TopDirector_" + str(i) for i in range(6)]

In [35]:
df1 = df1.withColumn('TopActor', greatest(df1['TopActor_0'],df1['TopActor_1'],df1['TopActor_2'],df1['TopActor_3'],
                                             df1['TopActor_4'],df1['TopActor_5'],df1['TopActor_6'],df1['TopActor_7'],
                                             df1['TopActor_8'],df1['TopActor_9']))

df1 = df1.withColumn('TopDirector', greatest(df1['TopDirector_0'],df1['TopDirector_1'],df1['TopDirector_2'],
                                             df1['TopDirector_3'],df1['TopDirector_4'],df1['TopDirector_5']))

In [36]:
df1.groupby('TopActor').agg(F.count('TopActor').alias('count')).show()

+--------+--------+
|TopActor|   count|
+--------+--------+
|    null|       0|
|       1|28565914|
|       0|49516326|
+--------+--------+



In [37]:
df1.groupby('TopDirector').agg(F.count('TopDirector').alias('count')).show()

+-----------+--------+
|TopDirector|   count|
+-----------+--------+
|       null|       0|
|          1|12906815|
|          0|63000408|
+-----------+--------+



In [39]:
df1 = df1.drop(*act_col)
df1 = df1.drop(*dir_col)

In [40]:
df1.limit(5).toPandas()

,Movie_Id,Cust_Id,Rating,tconst,Year,Name,primaryTitle,titleType,runtimeMinutes,startYear,...,Act_Cri_Thr_Myst_Hor,Ad_Fan_Sci_Ani,Bio_War_Hist_Doc,Mus_Musi_Fam,Noir_West,Ad_Rom,Drama_n,Comedy_n,TopActor,TopDirector
0,7706,770129,4.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,1,0,0,0,0,0,1,0,1,0
1,7706,1931185,5.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,1,0,0,0,0,0,1,0,1,0
2,7706,250166,5.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,1,0,0,0,0,0,1,0,1,0
3,7706,328654,3.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,1,0,0,0,0,0,1,0,1,0
4,7706,2414873,4.0,tt0090022,1985,Silverado,Silverado,movie,133.0,1985,...,1,0,0,0,0,0,1,0,1,0


In [41]:
## Write to Hive tables

df1.write.mode('overwrite').saveAsTable("akarshsahu.processed_dataset_v3")
df1.write.mode('overwrite').saveAsTable("aghose.processed_dataset_v3")
df1.write.mode('overwrite').saveAsTable("mwehr.processed_dataset_v3")

## CUSTOMER LEVEL DATASET FOR CLUSTERING

In [42]:
## Read the hive table back
df2 = hive_context.table("akarshsahu.processed_dataset_v3")

In [43]:
## Check for duplicates at Customer/Movie Level -- No Duplicates found - Great News!

df2.\
    groupby(['Cust_Id', 'Movie_Id']).count().\
    where('count > 1').\
    sort('count', ascending=False).\
    show()

+-------+--------+-----+
|Cust_Id|Movie_Id|count|
+-------+--------+-----+
+-------+--------+-----+



In [130]:
newdf1.select('Top100Actor').distinct().show()

+-----------+
|Top100Actor|
+-----------+
|       null|
|          1|
|          0|
+-----------+



In [48]:
df2.columns

['Movie_Id',
 'Cust_Id',
 'Rating',
 'tconst',
 'Year',
 'Name',
 'primaryTitle',
 'titleType',
 'runtimeMinutes',
 'startYear',
 'isAdult',
 'genres',
 'averageRating',
 'numVotes',
 'Actor_0',
 'Actor_1',
 'Actor_2',
 'Actor_3',
 'Actor_4',
 'Actor_5',
 'Actor_6',
 'Actor_7',
 'Actor_8',
 'Actor_9',
 'Director_0',
 'Director_1',
 'Director_2',
 'Director_3',
 'Director_4',
 'Director_5',
 'genres_split',
 'Act_Cri_Thr_Myst_Hor',
 'Ad_Fan_Sci_Ani',
 'Bio_War_Hist_Doc',
 'Mus_Musi_Fam',
 'Noir_West',
 'Ad_Rom',
 'Drama_n',
 'Comedy_n',
 'TopActor',
 'TopDirector']

In [79]:
from pyspark.sql.functions import col,when
newdf = df2.withColumn("valid", when(col("Act_Cri_Thr_Myst_Hor") == 1, col("Act_Cri_Thr_Myst_Hor") * col("Rating"))
          .otherwise('null'))

In [88]:
df2.groupby('Cust_Id').\
    agg(F.avg(when(col('Drama_n') == 1, col('Rating'))).alias('avg_drama_rating')).show()

+-------+------------------+
|Cust_Id|      total_movies|
+-------+------------------+
|2612555|3.6030534351145036|
|1840869| 4.157894736842105|
|1962728| 3.198019801980198|
|1277056|              3.55|
|1992980|3.7666666666666666|
|2228253| 3.709342560553633|
| 132406|3.7468354430379747|
|2046020|             3.825|
|1147874|               4.2|
|1660284| 3.642857142857143|
|1023029|3.9661016949152543|
| 356454| 3.711864406779661|
|1547377|3.9655172413793105|
|2372856|3.5930232558139537|
|1398928|3.6983471074380163|
| 202242| 4.391304347826087|
|1628970|3.6987951807228914|
|1214391|3.9583333333333335|
|1928182|3.6176470588235294|
|2030442|2.7058823529411766|
+-------+------------------+
only showing top 20 rows



In [99]:
## Test for First customer 2612555 if the logic is properly filtering out for just drama movies first and then calculating average
testdf = df2.filter(col("Cust_Id") == 2612555).toPandas()
testdf[testdf.Drama_n == 1].Rating.mean()

## Logic is correct

3.6030534351145036

In [107]:
# Create Customer Level Data through aggregation and conditions

df_cust = df2.groupby('Cust_Id').\
            agg(F.count('*').alias('total_movies'),
                (F.sum('TopActor')/F.count('*')).alias('top_actor_movies'),
                (F.sum('TopDirector')/F.count('*')).alias('top_director_movies'),
                (F.sum('isAdult')/F.count('*')).alias('total_adultage_movies'),
                (F.sum('Act_Cri_Thr_Myst_Hor')/F.count('*')).alias('total_action_movies'),
                (F.sum('Ad_Fan_Sci_Ani')/F.count('*')).alias('total_scifi_movies'),
                (F.sum('Bio_War_Hist_Doc')/F.count('*')).alias('total_history_movies'),
                (F.sum('Mus_Musi_Fam')/F.count('*')).alias('total_musical_movies'),
                (F.sum('Ad_Rom')/F.count('*')).alias('total_romance_movies'),
                (F.sum('Drama_n')/F.count('*')).alias('total_drama_movies'),
                (F.sum('Noir_West')/F.count('*')).alias('total_noir_movies'),
                (F.sum('Comedy_n')/F.count('*')).alias('total_comedy_movies'),
                F.avg("Rating").alias('avg_overall_rating'),
                F.avg(when(col('Act_Cri_Thr_Myst_Hor') == 1, col('Rating'))).alias('avg_action_rating'),
                F.avg(when(col('Ad_Fan_Sci_Ani') == 1, col('Rating'))).alias('avg_scifi_rating'),
                F.avg(when(col('Bio_War_Hist_Doc') == 1, col('Rating'))).alias('avg_history_rating'),
                F.avg(when(col('Mus_Musi_Fam') == 1, col('Rating'))).alias('avg_musical_rating'),
                F.avg(when(col('Ad_Rom') == 1, col('Rating'))).alias('avg_romance_rating'),
                F.avg(when(col('Drama_n') == 1, col('Rating'))).alias('avg_drama_rating'),
                F.avg(when(col('Noir_West') == 1, col('Rating'))).alias('avg_noir_rating'),
                F.avg(when(col('Comedy_n') == 1, col('Rating'))).alias('avg_comedy_rating'),
                F.avg("runtimeMinutes").alias('avg_runtime'))

In [108]:
df_cust.limit(2).toPandas()

,Cust_Id,total_movies,top_actor_movies,top_director_movies,total_adultage_movies,total_action_movies,total_scifi_movies,total_history_movies,total_musical_movies,total_romance_movies,...,avg_overall_rating,avg_action_rating,avg_scifi_rating,avg_history_rating,avg_musical_rating,avg_romance_rating,avg_drama_rating,avg_noir_rating,avg_comedy_rating,avg_runtime
0,56741,638,0.302508,0.203762,0.0,0.529781,0.311912,0.098746,0.062696,0.142633,...,4.199060,4.079882,4.291457,4.269841,4.225000,4.164835,4.274194,5.000000,4.245487,111.998428
1,2311863,1274,0.380691,0.143642,0.0,0.584772,0.313187,0.109105,0.080848,0.148352,...,3.477237,3.446980,3.458647,3.798561,3.281553,3.370370,3.572700,3.791667,3.306604,113.168239


In [ ]:
## Write to Hive tables

df_cust.write.mode('overwrite').saveAsTable("akarshsahu.processed_customer_dataset")
df_cust.write.mode('overwrite').saveAsTable("aghose.processed_customer_dataset")
df_cust.write.mode('overwrite').saveAsTable("mwehr.processed_customer_dataset")